In [1]:
import os

import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from matplotlib import cm, transforms
from mpl_axes_aligner import align
import numpy as np
np.seterr(divide='ignore')
import pandas as pd
from tqdm import tqdm
from scipy import special
from scipy.stats import norm
from scipy.stats import multivariate_normal
import h5py
import torch
from torchviz import make_dot

import wf_func as wff

In [2]:
def normcombine(x, m, s, a):
    return a[0] * norm.pdf((x - m[0]) / s[0]) + a[1] * norm.pdf((x - m[1]) / s[1])

def normcombine2d(x, m, s, a, rho):
    return a[0, 0] * multivariate_normal.pdf(x, mean=[m[0, 0], m[1, 0]], cov=matrix(s[0, 0], s[1, 0], rho[0, 0])) + a[0, 1] * multivariate_normal.pdf(x, mean=[m[0, 0], m[1, 1]], cov=matrix(s[0, 0], s[1, 1], rho[0, 1])) + a[1, 0] * multivariate_normal.pdf(x, mean=[m[0, 1], m[1, 0]], cov=matrix(s[0, 1], s[1, 0], rho[1, 0])) + a[1, 1] * multivariate_normal.pdf(x, mean=[m[0, 1], m[1, 1]], cov=matrix(s[0, 1], s[1, 1], rho[1, 1]))

def matrix(sx, sy, rho):
    return np.array([[sx ** 2, rho * sx * sy], [rho * sx * sy, sy ** 2]])

def chargehist(t):
    c = norm.pdf(t, loc=160., scale=40.)
#     q1 = 150.8
#     sigma = 37.59
#     w = 2.433e-5
#     alpha = 0.01335
#     mu = 2.851e-5
#     c = np.exp(-mu)*(w*alpha*np.exp(-alpha*t))
#     c = c + mu*np.exp(-mu)*(
#         (1-w)/(sigma*np.sqrt(2*np.pi))*np.exp(-(t-q1)**2/(2*sigma**2))+
#         w*(alpha/2*np.exp(-alpha*(t-q1-alpha/2*sigma**2))*(1+special.erf(t-q1-alpha*sigma**2)/(np.sqrt(2)*sigma))))
    return c

In [3]:
std = 1.
spe_pre = wff.read_model('spe.h5', 1)
window = 1029
p = spe_pre[0]['parameters']
t = np.arange(window).astype(np.float)
fig = plt.figure()
# fig.tight_layout()
gs = gridspec.GridSpec(1, 1, figure=fig, left=0.15, right=0.85, top=0.95, bottom=0.15, wspace=0.4, hspace=0.5)
ax = fig.add_subplot(gs[0, 0])
t = np.arange(0, 100, 0.1)
ax.plot(t, wff.spe(t, p[0], p[1], p[2]), color='b', label='Single PE response')
ax.set_xlabel(r'$t/\mathrm{ns}$')
ax.grid()
ax.set_xlim(0, 80)
ax.set_ylabel(r'$Voltage/\mathrm{mV}$')
ax.legend()
fig.savefig('Note/figures/spe.pgf')
fig.savefig('Note/figures/spe.pdf')
plt.close()

In [4]:
Mu = 5
Tau = 40
Sigma = 10
file = '{0:02d}'.format(Mu) + '-' + str(Tau) + '-' + str(Sigma)
fig = plt.figure()
t = np.arange(-4 * int(Sigma), int(5 * Tau), 0.1)
gs = gridspec.GridSpec(1, 1, figure=fig, left=0.15, right=0.85, top=0.95, bottom=0.15, wspace=0.4, hspace=0.5)
ax = fig.add_subplot(gs[0, 0])
ax.plot(t, wff.convolve_exp_norm(t, Tau, Sigma), label='Time Profile')
ax.set_xlabel(r'$t/\mathrm{ns}$')
ax.grid()
ax.set_xlim(xmin=-4 * int(Sigma))
ax.set_ylabel(r'$PDF$')
ax.legend()
ax.set_ylim(0, ax.get_ylim()[1] * 1.05)
fig.savefig('Note/figures/profile.pgf')
fig.savefig('Note/figures/profile.pdf')
plt.close()

In [5]:
fig = plt.figure()
# fig.tight_layout()
gs = gridspec.GridSpec(1, 1, figure=fig, left=0.12, right=0.95, top=0.9, bottom=0.15, wspace=0.4, hspace=0.5)
ax = fig.add_subplot(gs[0, 0])
ax.plot(np.arange(500), chargehist(np.arange(500)), label='$\mathrm{ChargePDF}$')
ax.set_xlabel(r'$Charge/\mathrm{mV}\cdot\mathrm{ns}$')
ax.set_xlim(20, 400)
ax.set_ylim(0, ax.get_ylim()[1] * 1.05)
ax.set_ylabel(r'$PDF$')
ax.grid()
ax.legend()
fig.savefig('Note/figures/chargehist.pgf')
fig.savefig('Note/figures/chargehist.pdf')
fig.clf()
plt.close(fig)

In [6]:
xn = np.linspace(-5, 5, 600)
yn = np.linspace(-5, 5, 600)
xv, yv = np.meshgrid(xn, yn)
pos = np.dstack((xv, yv))

m = np.array([[-1., 1.5], [-1., 1.]])
s = np.array([[1., .5], [.6, .8]])
a = np.array([[.4, .6], [.4, .6]])
rho = np.array([[0.5, -0.7], [0.4, 0.4]])

ymax = max(np.max(normcombine(xn, m[0, :], s[0, :], a[0, :])), np.max(normcombine(yn, m[1, :], s[1, :], a[1, :])))
fig = plt.figure(figsize=(5, 5))
ax0 = fig.add_axes((.1, .85, .75, .12))
ax0.axis('off')
ax0.plot(xn, normcombine(xn, m[0, :], s[0, :], a[0, :]), color='b')
ax0.fill_between(xn, normcombine(xn, m[0, :], s[0, :], a[0, :]), facecolor='blue', alpha=0.5, color=None)
ax0.set_xticks([])
ax0.set_yticks([])
ax0.set_ylim(0, ymax)
ax1 = fig.add_axes((.85, .1, .12, .75))
rot = transforms.Affine2D().rotate_deg(-90)
base = plt.gca().transData
ax1.axis('off')
ax1.plot(yn, normcombine(yn, m[1, :], s[1, :], a[1, :]), color='b', transform=rot+base)
ax1.fill_between(yn, normcombine(yn, m[1, :], s[1, :], a[1, :]), facecolor='blue', alpha=0.5, color=None, transform=rot+base)
ax1.set_xticks([])
ax1.set_yticks([])
ax1.set_xlim(0, ymax)
ax2 = fig.add_axes((.1, .1, .75, .75))
ax2.contourf(xv, yv, normcombine2d(pos, m, s, a, rho), cmap=cm.PuBu)
ax2.set_xticks([])
ax2.set_yticks([])
ax2.set_xlabel(r'$\mu$')
ax2.set_ylabel(r'$\nu$')
ax2.spines['top'].set_visible(False)
ax2.spines['right'].set_visible(False)
fig.savefig('Note/figures/wdistdemo.pgf')
fig.savefig('Note/figures/wdistdemo.pdf')
fig.clf()
plt.close(fig)

In [7]:
t = np.array([0, 1, 2, 3])
a = np.array([0.5, 0.5, 0.5, 1])
b1 = np.array([0.5, 0.5, 1, 0.5])
b2 = np.array([1, 0.5, 0.5, 0.5])
fig = plt.figure()
# fig.tight_layout()
gs = gridspec.GridSpec(2, 2, figure=fig, left=0.12, right=0.95, top=0.97, bottom=0.15, wspace=0.4, hspace=0.5)
ax0 = fig.add_subplot(gs[0, 0])
ax1 = fig.add_subplot(gs[0, 1])
ax2 = fig.add_subplot(gs[1, 0])
ax3 = fig.add_subplot(gs[1, 1])
ax0.bar(t, a, color='r')
ax0.set_xlabel(r'$a$')
ax0.set_ylabel(r'$weight\ a$')
ax0.set_xticks(t)
ax0.set_ylim(0, ax0.get_ylim()[1] * 1.05)
ax1.bar(t, b1, color='b')
ax1.set_xlabel(r'$b_{1}$')
ax1.set_ylabel(r'$weight\ b_{1}$')
ax1.set_xticks(t)
ax1.set_ylim(0, ax1.get_ylim()[1] * 1.05)
ax2.bar(t, b2, color='b')
ax2.set_xlabel(r'$b_{2}$')
ax2.set_ylabel(r'$weight\ b_{2}$')
ax2.set_xticks(t)
ax2.set_ylim(0, ax2.get_ylim()[1] * 1.05)
ax3.text(0, 0, r'$L_{2}\ =\ 0.25$')
ax3.set_xlim(0, 5)
ax3.set_ylim(-1, 1)
ax3.spines['top'].set_visible(False)
ax3.spines['right'].set_visible(False)
ax3.spines['bottom'].set_visible(False)
ax3.spines['left'].set_visible(False)
ax3.get_xaxis().set_visible(False)
ax3.get_yaxis().set_visible(False)
fig.savefig('Note/figures/tab.pgf')
fig.savefig('Note/figures/tab.pdf')
fig.clf()
plt.close(fig)

In [8]:
with h5py.File('waveform/' + file + '.h5', 'r', libver='latest', swmr=True) as ipt:
    ent = ipt['Readout/Waveform'][:]
    tru = ipt['SimTriggerInfo/PEList'][:]
    gmu = ipt['SimTriggerInfo/PEList'].attrs['gmu']
    gsigma = ipt['SimTriggerInfo/PEList'].attrs['gsigma']
    npe = ipt['SimTruth/T'].attrs['npe']

In [9]:
i = 2
cid = ent[i]['ChannelID']
eid = ent[i]['TriggerNo']
truth = np.sort(tru[(tru['TriggerNo'] == eid) & (tru['PMTId'] == cid)], kind='stable', order=['TriggerNo', 'PMTId', 'HitPosInWindow'])
wave = ent[i]['Waveform'].astype(np.float) * spe_pre[ent[i]['ChannelID']]['epulse']
df = pd.DataFrame(truth)
df = df.rename(columns={'HitPosInWindow':'HitTime'})
charge = df['Charge'].copy()
hittime = df['HitTime'].copy()
df = df.astype({'Charge': 'float32'})
df = df.astype({'TriggerNo' : 'str', 'PMTId' : 'str', 'HitTime' : 'str', 'Charge': 'str'})
df['HitTime'] = ['{:.02f}'.format(s) for s in hittime]
df['Charge'] = ['{:.02f}'.format(s) for s in charge]
df

,TriggerNo,PMTId,HitTime,Charge
0,2,0,371.33,184.44
1,2,0,381.91,223.93
2,2,0,397.86,164.65
3,2,0,426.23,110.10
4,2,0,427.07,159.53
5,2,0,444.86,117.11


In [10]:
ind = np.argwhere(wave > spe_pre[cid]['std'] * 5).flatten()
xmin = ((ind.min() - spe_pre[cid]['mar_l']) // 50 - 1) * 50
xmax = max(((ind.max() + spe_pre[cid]['mar_r']) // 50 + 1) * 50, xmin + 400)

In [11]:
fig = plt.figure()
# fig.tight_layout()
gs = gridspec.GridSpec(1, 1, figure=fig, left=0.15, right=0.85, top=0.95, bottom=0.15, wspace=0.4, hspace=0.5)
ax = fig.add_subplot(gs[0, 0])
ax2 = ax.twinx()
ax2.vlines(truth['HitPosInWindow'].min(), 0, wave.sum(), color='r', label='Record')
ax.plot(wave, label='Waveform')
ax.hlines(2, 0, window, color='g', label='Threshold')
ax.set_xlim(xmin, xmax)
lines, labels = ax.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
ax2.legend(lines + lines2, labels + labels2)
ax.set_xlabel(r'$t/\mathrm{ns}$')
ax.set_ylabel(r'$Voltage/\mathrm{mV}$')
ax2.set_ylabel(r'$Charge/\mathrm{mV}\cdot\mathrm{ns}$')
rise = truth['HitPosInWindow'].min()
vsum = wave.sum()
ax2.annotate(fr'${rise:.{4}}\mathrm{{ns}}, {vsum:.{4}}\mathrm{{mV}}\cdot\mathrm{{ns}}$', xy=(truth['HitPosInWindow'].min(), wave.sum()*2/3), xytext=(truth['HitPosInWindow'].min()+120, wave.sum()*1/2), arrowprops=dict(facecolor='k', shrink=0.1, width=0.1, headwidth=2))
ax.set_ylim(-5, ax.get_ylim()[1] * 1.05)
ax2.set_ylim(-5, ax2.get_ylim()[1] * 1.05)
align.yaxes(ax, 0, ax2, 0)
fig.savefig('Note/figures/previous.pgf')
fig.savefig('Note/figures/previous.pdf')
fig.clf()
plt.close(fig)

In [12]:
fig = plt.figure()
# fig.tight_layout()
gs = gridspec.GridSpec(1, 1, figure=fig, left=0.15, right=0.85, top=0.95, bottom=0.15, wspace=0.4, hspace=0.5)
ax = fig.add_subplot(gs[0, 0])
ax.plot(wave, label='Waveform')
ax.set_xlabel(r'$t/\mathrm{ns}$')
ax.set_ylabel(r'$Voltage/\mathrm{mV}$')
ax.set_xlim(0, len(wave))
ax.set_ylim(-5, ax.get_ylim()[1] * 1.05)
ax.legend()
fig.savefig('Note/figures/wave.pgf')
fig.savefig('Note/figures/wave.pdf')
fig.clf()
plt.close(fig)

fig = plt.figure()
# fig.tight_layout()
gs = gridspec.GridSpec(1, 1, figure=fig, left=0.15, right=0.85, top=0.95, bottom=0.15, wspace=0.4, hspace=0.5)
ax = fig.add_subplot(gs[0, 0])
ax.vlines(truth['HitPosInWindow'], 0, truth['Charge'], color='r', label='Charge')
ax.set_xlabel(r'$t/\mathrm{ns}$')
ax.set_ylabel(r'$Charge/\mathrm{mV}\cdot\mathrm{ns}$')
ax.set_xlim(0, len(wave))
ax.legend()
fig.savefig('Note/figures/charge.pgf')
fig.savefig('Note/figures/charge.pdf')
fig.clf()
plt.close(fig)

In [13]:
fig = plt.figure()
# fig.tight_layout()
gs = gridspec.GridSpec(1, 1, figure=fig, left=0.15, right=0.85, top=0.95, bottom=0.15, wspace=0.4, hspace=0.5)
ax = fig.add_subplot(gs[0, 0])
ax2 = ax.twinx()
ax2.vlines(truth['HitPosInWindow'], 0, truth['Charge'], color='r', label='Charge')
ax.plot(wave, label='Waveform')
ax.hlines(2, 0, window, color='g', label='Threshold')
ax.set_xlim(xmin, xmax)
lines, labels = ax.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
ax2.legend(lines + lines2, labels + labels2)
ax.set_xlabel(r'$t/\mathrm{ns}$')
ax.set_ylabel(r'$Voltage/\mathrm{mV}$')
ax2.set_ylabel(r'$Charge/\mathrm{mV}\cdot\mathrm{ns}$')
ax.set_ylim(bottom=-5)
ax2.set_ylim(bottom=-5)
align.yaxes(ax, 0, ax2, 0)
fig.savefig('Note/figures/goal.pgf')
fig.savefig('Note/figures/goal.pdf')
fig.clf()
plt.close(fig)

In [14]:
print(wave.sum())
print(truth['Charge'][truth['Charge'] > 0].sum()) # made by noise
truth['Charge']

1005.0
959.7484134846668


array([184.44219163, 223.9269568 , 164.64571677, 110.09763184,
       159.52583068, 117.11008577])

In [15]:
t = np.load('result/takara/char/Channel00/testing_record_2021-03-05_02:54:28.npz')['arr_0']
fig = plt.figure()
# fig.tight_layout()
gs = gridspec.GridSpec(1, 1, figure=fig, left=0.15, right=0.85, top=0.95, bottom=0.15, wspace=0.4, hspace=0.5)
ax = fig.add_subplot(gs[0, 0])
ax.plot(np.arange(1, len(t)+1), t, label='W-distance', color='C1')
ax.set_xlabel(r'$epoch$')
ax.set_ylabel(r'$Wasserstein\ Distance/\mathrm{ns}$')
ax.legend()
ax.grid()
fig.savefig('Note/figures/epoch.pgf')
fig.savefig('Note/figures/epoch.pdf')
fig.clf()
plt.close(fig)

In [16]:
pet, pwe = wff.threshold(wave, spe_pre[cid])
pet, pwe = wff.clip(pet, pwe, 0.1)
pwe = pwe / pwe.sum() * np.abs(wave.sum())
fig = plt.figure()
# fig.tight_layout()
ax = fig.add_subplot(111)
ax2 = ax.twinx()
ax2.vlines(pet, 0, pwe, color='r', label='Charge', linewidth=0.5)
ax.plot(wave, label='Waveform')
ax.hlines(5 * spe_pre[cid]['std'], 0, window, color='g', label='Threshold')
ax.set_xlim(xmin, xmax)
ax2.annotate('', xy=(pet.mean(), pwe.max()*1.1), xytext=(pet.mean()+pet.ptp(), pwe.max()*1.1), arrowprops=dict(facecolor='k', shrink=0.01, width=2, headwidth=4))
ax2.set_ylim(top=pwe.max()*1.2)
lines, labels = ax.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
ax2.legend(lines + lines2, labels + labels2)
ax.set_xlabel(r'$t/\mathrm{ns}$')
ax.set_ylabel(r'$Voltage/\mathrm{mV}$')
ax2.set_ylabel(r'$Charge/\mathrm{mV}\cdot\mathrm{ns}$')
align.yaxes(ax, 0, ax2, 0)
fig.savefig('Note/figures/threshold.pgf')
fig.savefig('Note/figures/threshold.pdf')
fig.clf()
plt.close(fig)
wff.demo(pet, pwe, truth, spe_pre[cid], window, wave, cid, p)

PEnum is 6
truth HitPosInWindow = [371.33044571 381.90834032 397.86129548 426.22550161 427.06543306
 444.86196172], Weight = [184.44219163 223.9269568  164.64571677 110.09763184 159.52583068
 117.11008577]
truth Resi-norm = 32.18761855427509
HitPosInWindow = [370 371 372 373 378 379 380 381 382 383 384 385 386 387 396 397 398 399
 400 424 425 426 427 428 429 430 431 432], Weight = [30.6870229  32.60496183 30.6870229  30.6870229  30.6870229  40.27671756
 42.19465649 44.11259542 47.94847328 42.19465649 38.35877863 34.52290076
 32.60496183 28.76908397 28.76908397 32.60496183 34.52290076 30.6870229
 28.76908397 32.60496183 42.19465649 42.19465649 44.11259542 42.19465649
 40.27671756 36.44083969 32.60496183 30.6870229 ]
wdist = 5.576416991336991, cdiff = 45.251586515332974
Resi-norm = 55.34605069895723


In [17]:
pet, pwe = wff.findpeak(wave, spe_pre[cid])
pet, pwe = wff.clip(pet, pwe, 0.1)
pwe = pwe / pwe.sum() * np.abs(wave.sum())
fig = plt.figure()
# fig.tight_layout()
ax = fig.add_subplot(111)
ax2 = ax.twinx()
ax2.vlines(pet, 0, pwe, color='r', label='Charge', linewidth=1.5)
ax.plot(wave, label='Waveform')
ax.hlines(5 * spe_pre[cid]['std'], 0, window, color='g', label='Threshold')
ax.set_xlim(xmin, xmax)
loc = pet + spe_pre[cid]['peak_c']
loc = loc[loc < window]
amp = wave[loc]
for i in range(len(loc)):
    ax.annotate('', xy=(loc[i], amp[i]+5), xytext=(loc[i], amp[i]+15), arrowprops=dict(facecolor='k', shrink=0.01, width=0.5, headwidth=2))
ax2.annotate('', xy=(pet.mean(), pwe.max()*1.1), xytext=(pet.mean()+pet.ptp(), pwe.max()*1.1), arrowprops=dict(facecolor='k', shrink=0.01, width=2, headwidth=4))
ax2.set_ylim(top=pwe.max()*1.2)
lines, labels = ax.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
ax2.legend(lines + lines2, labels + labels2)
ax.set_xlabel(r'$t/\mathrm{ns}$')
ax.set_ylabel(r'$Voltage/\mathrm{mV}$')
ax2.set_ylabel(r'$Charge/\mathrm{mV}\cdot\mathrm{ns}$')
align.yaxes(ax, 0, ax2, 0)
fig.savefig('Note/figures/findpeak.pgf')
fig.savefig('Note/figures/findpeak.pdf')
fig.clf()
plt.close(fig)
wff.demo(pet, pwe, truth, spe_pre[cid], window, wave, cid, p)

PEnum is 6
truth HitPosInWindow = [371.33044571 381.90834032 397.86129548 426.22550161 427.06543306
 444.86196172], Weight = [184.44219163 223.9269568  164.64571677 110.09763184 159.52583068
 117.11008577]
truth Resi-norm = 32.18761855427509
HitPosInWindow = [372 383 399 428 446], Weight = [182.72727273 251.25       182.72727273 251.25       137.04545455]
wdist = 1.9575739688210545, cdiff = 45.2515865153332
Resi-norm = 39.70439846903642


In [18]:
pet, pwe = wff.waveformfft(wave, spe_pre[cid])
pet, pwe = wff.clip(pet, pwe, 0.1)
pwe = pwe / pwe.sum() * np.abs(wave.sum())
fig = plt.figure()
# fig.tight_layout()
ax = fig.add_subplot(111)
ax2 = ax.twinx()
ax2.vlines(pet, 0, pwe, color='r', label='Charge', linewidth=0.5)
ax.plot(wave, label='Waveform')
ax.hlines(5 * spe_pre[cid]['std'], 0, window, color='g', label='Threshold')
ax.set_xlim(xmin, xmax)
lines, labels = ax.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
ax2.legend(lines + lines2, labels + labels2)
ax.set_xlabel(r'$t/\mathrm{ns}$')
ax.set_ylabel(r'$Voltage/\mathrm{mV}$')
ax2.set_ylabel(r'$Charge/\mathrm{mV}\cdot\mathrm{ns}$')
align.yaxes(ax, 0, ax2, 0)
fig.savefig('Note/figures/fftrans.pgf')
fig.savefig('Note/figures/fftrans.pdf')
fig.clf()
plt.close(fig)
wff.demo(pet, pwe, truth, spe_pre[cid], window, wave, cid, p)

PEnum is 6
truth HitPosInWindow = [371.33044571 381.90834032 397.86129548 426.22550161 427.06543306
 444.86196172], Weight = [184.44219163 223.9269568  164.64571677 110.09763184 159.52583068
 117.11008577]
truth Resi-norm = 32.18761855427509
HitPosInWindow = [368 369 370 371 372 373 380 381 382 383 384 385 394 395 396 397 398 399
 400 423 424 425 426 427 428 429 430 443 444 445 446 447], Weight = [20.67250235 30.40003361 36.78460491 38.22636503 34.33829265 26.14170436
 27.8166618  39.11140084 46.38399781 47.56315413 42.07412342 30.97391267
 17.64879995 25.23576626 29.59209333 30.63603968 28.93235268 25.3666667
 20.81741315 22.70955846 34.13505303 43.01812905 47.84059032 47.91142999
 43.46649992 35.53806981 25.6492316  18.84249621 22.07550961 23.40104434
 22.44934644 19.24715589]
wdist = 1.7677444778030196, cdiff = 45.251586515333315
Resi-norm = 35.802598165692636


In [19]:
pet, pwe = wff.lucyddm(wave, spe_pre[cid]['spe'])
pet, pwe = wff.clip(pet, pwe, 0.1)
pwe = pwe / pwe.sum() * np.abs(wave.sum())
fig = plt.figure()
# fig.tight_layout()
ax = fig.add_subplot(111)
ax2 = ax.twinx()
ax2.vlines(pet, 0, pwe, color='r', label='Charge', linewidth=0.5)
ax.plot(wave, label='Waveform')
ax.hlines(5 * spe_pre[cid]['std'], 0, window, color='g', label='Threshold')
ax.set_xlim(xmin, xmax)
lines, labels = ax.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
ax2.legend(lines + lines2, labels + labels2)
ax.set_xlabel(r'$t/\mathrm{ns}$')
ax.set_ylabel(r'$Voltage/\mathrm{mV}$')
ax2.set_ylabel(r'$Charge/\mathrm{mV}\cdot\mathrm{ns}$')
align.yaxes(ax, 0, ax2, 0)
fig.savefig('Note/figures/lucyddm.pgf')
fig.savefig('Note/figures/lucyddm.pdf')
fig.clf()
plt.close(fig)
wff.demo(pet, pwe, truth, spe_pre[cid], window, wave, cid, p)

PEnum is 6
truth HitPosInWindow = [371.33044571 381.90834032 397.86129548 426.22550161 427.06543306
 444.86196172], Weight = [184.44219163 223.9269568  164.64571677 110.09763184 159.52583068
 117.11008577]
truth Resi-norm = 32.18761855427509
HitPosInWindow = [371 372 381 382 383 397 398 399 425 426 427 428 444 445 446], Weight = [129.66609554  65.41156666  68.39413877 103.71371864  53.53479043
  43.09271713  75.67260381  45.77040891  29.44746574  89.28314196
 123.2340827   48.78263276  36.04775467  67.54027474  25.40860753]
wdist = 1.0995216012043345, cdiff = 45.251586515332974
Resi-norm = 32.6021378793333


In [20]:
pet, pwe = wff.xiaopeip(wave, spe_pre[cid])
pet, pwe = wff.clip(pet, pwe, 0.1)
pwe = pwe / pwe.sum() * np.abs(wave.sum())
fig = plt.figure()
# fig.tight_layout()
ax = fig.add_subplot(111)
ax2 = ax.twinx()
ax2.vlines(pet, 0, pwe, color='r', label='Charge', linewidth=0.5)
ax.plot(wave, label='Waveform')
ax.hlines(5 * spe_pre[cid]['std'], 0, window, color='g', label='Threshold')
ax.set_xlim(xmin, xmax)
lines, labels = ax.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
ax2.legend(lines + lines2, labels + labels2)
ax.set_xlabel(r'$t/\mathrm{ns}$')
ax.set_ylabel(r'$Voltage/\mathrm{mV}$')
ax2.set_ylabel(r'$Charge/\mathrm{mV}\cdot\mathrm{ns}$')
align.yaxes(ax, 0, ax2, 0)
fig.savefig('Note/figures/xiaopeip.pgf')
fig.savefig('Note/figures/xiaopeip.pdf')
fig.clf()
plt.close(fig)
wff.demo(pet, pwe, truth, spe_pre[cid], window, wave, cid, p)

PEnum is 6
truth HitPosInWindow = [371.33044571 381.90834032 397.86129548 426.22550161 427.06543306
 444.86196172], Weight = [184.44219163 223.9269568  164.64571677 110.09763184 159.52583068
 117.11008577]
truth Resi-norm = 32.18761855427509
HitPosInWindow = [371 372 381 382 398 426 427 445], Weight = [169.07995244  29.09775888  21.04642501 219.39641861 168.29776276
  80.26531646 190.95746322 126.85890262]
wdist = 0.6911397195718068, cdiff = 45.251586515333315
Resi-norm = 32.39204726253358


In [21]:
spe_pre_fbmp = wff.read_model('spe.h5', wff.nshannon)
p = spe_pre_fbmp[cid]['parameters']
nsp = 4
nstd = 3
D = 20
factor = np.linalg.norm(spe_pre_fbmp[cid]['spe'])
A, wave_r, tlist, t0_init, t0_init_delta, char_init, n = wff.initial_params(wff.shannon_interpolation(wave, wff.nshannon), spe_pre_fbmp[cid], Mu, Tau, Sigma, gmu, gsigma, 0.2, npe, p, nsp, nstd, is_t0=True, is_delta=False, nshannon=wff.nshannon)
A = A / factor
xmmse, xmmse_star, psy_star, nu_star, T_star, d_tot_i, d_max_i = wff.fbmpr_fxn_reduced(wave_r, A, min(-1e-3+1, Mu / len(tlist)), spe_pre_fbmp[cid]['std'] ** 2, (gsigma * factor / gmu) ** 2, factor, D, stop=0)
xmmse_most = xmmse_star[0]
pet = tlist[xmmse_most > 0]
cha = xmmse_most[xmmse_most > 0] / factor
pet, pwe = wff.clip(pet, cha, 0.0)
pwe = pwe * gmu
fig = plt.figure()
# fig.tight_layout()
ax = fig.add_subplot(111)
ax2 = ax.twinx()
ax2.vlines(pet, 0, pwe, color='r', label='Charge', linewidth=0.5)
ax.plot(wave, label='Waveform')
ax.hlines(5 * spe_pre_fbmp[cid]['std'], 0, window, color='g', label='Threshold')
ax.set_xlim(xmin, xmax)
lines, labels = ax.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
ax2.legend(lines + lines2, labels + labels2)
ax.set_xlabel(r'$t/\mathrm{ns}$')
ax.set_ylabel(r'$Voltage/\mathrm{mV}$')
ax2.set_ylabel(r'$Charge/\mathrm{mV}\cdot\mathrm{ns}$')
align.yaxes(ax, 0, ax2, 0)
fig.savefig('Note/figures/fbmp.pgf')
fig.savefig('Note/figures/fbmp.pdf')
fig.clf()
plt.close(fig)
wff.demo(pet, pwe, truth, spe_pre_fbmp[cid], window, wave, cid, p, n=wff.nshannon)

PEnum is 6
truth HitPosInWindow = [371.33044571 381.90834032 397.86129548 426.22550161 427.06543306
 444.86196172], Weight = [184.44219163 223.9269568  164.64571677 110.09763184 159.52583068
 117.11008577]
truth Resi-norm = 32.187623720192505
HitPosInWindow = [371. 381. 382. 398. 426. 427. 445.], Weight = [185.26713368  34.75037968 196.04184398 168.39346149  88.78143674
 180.629787   124.12368956]
wdist = 0.33181426731928687, cdiff = 18.23931865367001
Resi-norm = 32.10434222922358


In [22]:
with h5py.File('result/threshold/dist/' + file + '.h5', 'r', libver='latest', swmr=True) as distfile:
    dt = distfile['Record'][:]
fig = plt.figure()
# fig.tight_layout()
gs = gridspec.GridSpec(1, 1, figure=fig, left=0.15, right=0.85, top=0.95, bottom=0.15, wspace=0.4, hspace=0.5)
ax = fig.add_subplot(gs[0, 0])
ax.hist(dt['NPE'], bins=np.max(dt['NPE']), histtype='step')
ax.set_yscale('log')
ax.grid()
ax.set_xlabel(r'$N_{pe}$')
ax.set_ylabel(r'$Count$')
fig.savefig('Note/figures/penum.pgf')
fig.savefig('Note/figures/penum.pdf')
plt.close(fig)

In [23]:
with h5py.File('result/mcmc/solu/' + file + '.h5', 'r', libver='latest', swmr=True) as soluf, h5py.File('waveform/' + file + '.h5', 'r', libver='latest', swmr=True) as wavef:
    start = wavef['SimTruth/T'][:]
    time = soluf['starttime'][:]
data = time['tswave'] - start['T0']
fig = plt.figure()
ax = fig.add_axes((.1, .45, .85, .45))
ax.hist(data, bins=100, density=1)
ax.set_ylabel(r'$Normalized\ Count$')
ax.set_xticks([])
ax.set_yticks([])
ax.set_xlim(data.min() - 0.05, data.max() + 0.05)
ax.set_ylim(0, ax.get_ylim()[1] * 1.05)
axb = fig.add_axes((.1, .15, .85, .3))
axb.boxplot(data, vert=False, sym='', patch_artist=True)
axb.set_xlabel(r'$\Delta t_{0}/\mathrm{ns}$')
axb.set_yticks([])
axb.set_xlim(ax.get_xlim())
fig.savefig('Note/figures/mcmcrect0hist.pgf')
fig.savefig('Note/figures/mcmcrect0hist.pdf')
plt.close(fig)

In [26]:
methods = ['lucyddm', 'xiaopeip', 'takara', 'fbmp', 'mcmc']

for m in methods:
    with h5py.File('result/' + m + '/dist/' + file + '.h5', 'r', libver='latest', swmr=True) as distfile:
        dt = distfile['Record'][:]
    N = np.percentile(dt['wdist'], 95)
    M = 500

    penum = np.unique(dt['NPE'])
    l = min(50, penum.max())
    wdist_stats = np.full((l, 6), np.nan)
    edist_stats = np.full((l, 6), np.nan)
    for i in range(l):
        vali = dt['NPE'] == i+1
        if np.sum(vali) == 0:
            continue
        dtwpi = dt['wdist'][vali]
        dtepi = dt['RSS'][vali]
        wdist_stats[i, 0] = np.median(dtwpi)
        wdist_stats[i, 1] = np.median(np.abs(dtwpi - np.median(dtwpi)))
        wdist_stats[i, 2] = np.mean(dtwpi)
        wdist_stats[i, 3] = np.std(dtwpi)
        wdist_stats[i, 4] = np.percentile(dtwpi, 5)
        wdist_stats[i, 5] = np.percentile(dtwpi, 95)
        edist_stats[i, 0] = np.median(dtepi)
        edist_stats[i, 1] = np.median(np.abs(dtepi - np.median(dtepi)))
        edist_stats[i, 2] = np.mean(dtepi)
        edist_stats[i, 3] = np.std(dtepi)
        edist_stats[i, 4] = np.percentile(dtepi, 5)
        edist_stats[i, 5] = np.percentile(dtepi, 95)

#     n = 4 # corresponding with Algorithm.tex
    L = len(dt)
    fig = plt.figure()
    ax = fig.add_axes((.1, .45, .85, .45))
    data = dt['wdist']
    vali = np.abs(data - np.mean(data)) < 7 * np.std(data, ddof=-1)
    data = data[vali]
    ax.hist(data, bins=np.arange(0, data.max(), np.percentile(data, 95) / 50), density=1)
    ax.set_ylabel(r'$Normalized\ Count$')
    ax.set_xticks([])
    ax.set_yticks([])
    ax.set_xlim(data.min() - 0.05, np.percentile(data, 95))
    ax.set_ylim(0, ax.get_ylim()[1] * 1.05)
    axb = fig.add_axes((.1, .15, .85, .3))
    axb.boxplot(data, vert=False, sym='', patch_artist=True)
    axb.set_xlabel(r'$W-dist/\mathrm{ns}$')
    axb.set_yticks([])
    axb.set_xlim(ax.get_xlim())
    fig.savefig('Note/figures/' + m + 'chargehist.pgf')
    fig.savefig('Note/figures/' + m + 'chargehist.pdf')
    plt.close(fig)

    fig = plt.figure()
    gs = gridspec.GridSpec(1, 1, figure=fig, left=0.15, right=0.85, top=0.95, bottom=0.15, wspace=0.4, hspace=0.5)
    ax = fig.add_subplot(gs[0, 0])
#     ey = np.vstack([wdist_stats[:, 0]-wdist_stats[:, 4], wdist_stats[:, 5]-wdist_stats[:, 0]])
#     ax.errorbar(np.arange(1, l+1), wdist_stats[:, 0], yerr=ey, label=r'$W-dist^{95\%}_{5\%}$')
    ax.boxplot(np.array([dt['wdist'][dt['NPE'] == i+1] for i in range(l)], dtype=np.object), sym='', patch_artist=True)
    ax.plot(np.arange(1, l + 1), wdist_stats[:, 0], label=r'$W-dist$')
    ax.set_xlim(-1, l + 1)
    ax.set_xlabel(r'$N_{pe}$')
    ax.set_ylabel(r'$W-dist/\mathrm{ns}$')
    # ax.set_title('W-dist vs NPE stats')
    ax.legend()
    fig.savefig('Note/figures/' + m + 'chargestats.pgf')
    fig.savefig('Note/figures/' + m + 'chargestats.pdf')
    plt.close(fig)